In [1]:
import nltk
from nltk.corpus import wordnet#, stopwords
#stops = stopwords.words('english')
import spacy
from spacy.tokenizer import Tokenizer
import en_core_web_lg
nlp = en_core_web_lg.load()

In [13]:
# https://stackoverflow.com/questions/13928155/spell-checker-for-python/48280566
from autocorrect import Speller
spell = Speller(lang='en')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
tokenizer = Tokenizer(nlp.vocab)
import numpy as np

In [20]:
# Not super useful to take the mean across columns, instead look at top 10 scoring words in each side effect
def findTop(strList, keeptop=10):
    tfidf_vectr = TfidfVectorizer()
    corpus = [' '.join(SE) for SE in strList]
    tfidf_score = tfidf_vectr.fit_transform(corpus).toarray()
    features = np.array(tfidf_vectr.get_feature_names())
    
    words = []
    for row in tfidf_score:
        inds = row.argsort()[::-1][:keeptop]
        row_words = []
        for ind in inds:
            if row[ind] != 0:
                row_words.append(features[ind])
                #print(features[ind],' '*(50-len(features[ind])), row[ind].round(2))
        #print('\n')
        if not row_words:
            row_words = list(features[inds][:5])
        words.append(row_words)
    return tfidf_score, words

# Magic tokenizer thing
def spacyTokenizer(s: str)-> list:
    doc = tokenizer(s.lower().strip())
    tokens = []
    for token in doc:
        if not token.is_stop and token.is_alpha and token.lemma_ != '-PRON-':
            tokens.append(token.lemma_)
        
    return tokens

def parseRevnew(review):
    reviews = [review]
    clean_reviews = [spacyTokenizer(rev.replace('/', ' ')) for rev in reviews]
    feat_weights, clean_rev = findTop(clean_reviews, 50)
    cleaner_reviews = [[spell(word.lower()) for word in rev] for rev in clean_rev]
    
    return feat_weights, cleaner_reviews#consider, ignore

In [24]:
score, words = parseRevnew("...I'm hoping this really wierd drowzy/tired/sleepy/no motivation/no energy/no emotions/blah/netflix-binge mood is not how this medicine will make me feel for very lon. I didn't really notice any effects at 25mgs, but today, my first day on 50mgs has been rough. Had a panic attack earlier and then settled into this mood. Blllaaaahahhhhhhhh. :-/")

In [26]:
' '.join(words[0])


'wierd feel blah day drowsy early effect emotion energy hope tire medicine mood motivation notice panic settle sleepy attack'

In [ ]:
# Not super useful to take the mean across columns, instead look at top 10 scoring words in each side effect
def findTop(strList, keeptop=10):
    tfidf_vectr = TfidfVectorizer()
    corpus = [' '.join(SE) for SE in strList]
    tfidf_score = tfidf_vectr.fit_transform(corpus).toarray()
    features = np.array(tfidf_vectr.get_feature_names())
    
    words = []
    for row in tfidf_score:
        inds = row.argsort()[::-1][:keeptop]
        row_words = []
        for ind in inds:
            if row[ind] != 0:
                row_words.append(features[ind])
                #print(features[ind],' '*(50-len(features[ind])), row[ind].round(2))
        #print('\n')
        if not row_words:
            row_words = list(features[inds][:5])
        words.append(row_words)
    return tfidf_score, words

# Magic tokenizer thing
def spacyTokenizer(s: str)-> list:
    doc = tokenizer(s.lower().strip())
    tokens = []
    for token in doc:
        if not token.is_stop and token.is_alpha and token.lemma_ != '-PRON-':
            tokens.append(token.lemma_)
        
    return tokens

def parseRevnew(review):
    reviews = [review]
    clean_reviews = [spacyTokenizer(rev.replace('/', ' ')) for rev in reviews]
    feat_weights, clean_rev = findTop(clean_reviews, 50)
    cleaner_reviews = [[spell(word.lower()) for word in rev] for rev in clean_rev]
    
    return feat_weights, cleaner_reviews#consider, ignore